# 1. SETTING
- 환경 설정 및 모듈 import

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from datetime import date, timedelta

import pandas as pd
import numpy as np

In [3]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# 2. CRAWLING

In [5]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%84%9C%EC%9A%B8+%EB%82%A0%EC%94%A8' # 서울 중구 기준

In [6]:
driver.get(url)
time.sleep(5)

pageString = driver.page_source
soup = BeautifulSoup(pageString, features="html.parser")

In [7]:
week_list = soup.find_all(name='ul', attrs={'class': 'week_list'})

In [8]:
week_list_str = str(week_list)
week_list_str = week_list_str.split('<strong class="day">')[1:4]
week_list_str

['오늘</strong> <span class="date">12.03.</span> </span> </div> <div class="cell_weather"> <span class="weather_inner"> <span class="weather_left"> <strong class="time">오전</strong> <span class="rainfall">0%</span> </span> <i class="wt_icon ico_wt1"><span class="blind">맑음</span></i> </span> <span class="weather_inner"> <span class="weather_left"> <strong class="time">오후</strong> <span class="rainfall">0%</span> </span> <i class="wt_icon ico_wt1"><span class="blind">맑음</span></i> </span> </div> <div class="cell_temperature"> <span class="temperature_inner"> <span class="lowest"><span class="blind">최저기온</span>0°</span> <span class="bar">/</span> <span class="highest"><span class="blind">최고기온</span>4°</span> </span> </div> </div> </li> <li class="week_item today"> <div class="day_data"> <div class="cell_date"> <span class="date_inner"> ',
 '내일</strong> <span class="date">12.04.</span> </span> </div> <div class="cell_weather"> <span class="weather_inner"> <span class="weather_left"> <span cla

In [9]:
from datetime import date, datetime, timedelta

result = {}

for i in range(len(week_list_str)):
  whether_info = BeautifulSoup(week_list_str[i])
  
  # 일자
  today = date.today()
  new_date = today + timedelta(days=i)
  new_date = str(new_date.isoformat())

  highest = whether_info.find('span', attrs = {'class' : 'highest'}).text.split('최고기온')[-1].split('°')[0] # 최고기온
  lowest = whether_info.find('span', attrs = {'class' : 'lowest'}).text.split('최저기온')[-1].split('°')[0] # 최저기온
  
  # 최고기온설명, 최저기온설명
  wheter_info = []
  info = week_list_str[i].split('<span class="blind">')[1:3]
  for j in range(len(info)):
    wheter_info.append(info[j].split('<')[0])

  result[i] = [new_date, highest, wheter_info[1], lowest, wheter_info[0]]

whether_df = pd.DataFrame(result)
whether_df = whether_df.transpose()
whether_df.columns = ['일자', '최고기온', '최고기온설명', '최저기온', '최저기온설명']
whether_df

,일자,최고기온,최고기온설명,최저기온,최저기온설명
0,2021-12-03,4,맑음,0,맑음
1,2021-12-04,6,맑음,-3,구름많음
2,2021-12-05,8,맑음,-3,맑음


In [10]:
whether_df.to_csv('/content/drive/MyDrive/빅데이터통계분석/data/seoul_whether.csv', index = None)

from google.colab import files
files.download('/content/drive/MyDrive/빅데이터통계분석/data/seoul_whether.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>